In [1]:
# !pip install onnx

In [2]:
import torch
import sys
import os
os.environ["K_DIFFUSION_USE_COMPILE"] = "0"

import k_diffusion as K
import torch._C._onnx as _C_onnx
from torchinfo import summary
import cv2
import numpy as np
import torchvision

import pynvml
# import onnxruntime as ort
import os
import torch.nn as nn
from PIL import Image

In [3]:
torch.set_float32_matmul_precision('high')

In [4]:
config = "/home/ubuntu/tdist-flat/configs/hdit-shifted-windows/patchsize1.json"
config = K.config.load_config(config)

model = K.config.make_model(config).cuda()

# dct = torch.load("../latest_net_G.pth")#.keys()
# dct = {
#     key.replace('model.', ''): value for key, value in dct.items()
# }

# model.load_state_dict(dct)
final_activation_function = nn.Tanh()
print(sum([p.numel() for p in model.parameters()]) / 1e6)
model.eval()

# img = torch.load("/home/ubuntu/transformer-distillation/tensor_14.pt").unsqueeze(0).cuda()
img = torch.rand(1,3,1024,1024).cuda()

# raise ValueError(img.shape)

cst = torch.ones((img.shape[0])).cuda() * 500
with torch.no_grad(): out = final_activation_function(model(img, cst)[0])

out = out / 2. + .5
# raise ValueError(out.shape)
out = out.squeeze(0).transpose(0, 1).transpose(1, 2)
out = Image.fromarray((out.data.cpu().numpy() * 255.).astype(np.uint8))
out.save('out_shifted.png')

proj shape = torch.Size([128, 192]) 
 out_features = 128 
 other_dim = 192
proj shape = torch.Size([256, 512]) 
 out_features = 256 
 other_dim = 512
proj shape = torch.Size([384, 1024]) 
 out_features = 384 
 other_dim = 1024
proj shape = torch.Size([768, 1536]) 
 out_features = 768 
 other_dim = 1536
proj shape = torch.Size([1024, 3072]) 
 out_features = 1024 
 other_dim = 3072
102.212868


In [5]:
class MyModel(torch.nn.Module):
    def __init__(self, model, device):
        super(MyModel, self).__init__()
        # Example: simple linear layer
        self.cst = torch.ones((1)).to(device) *500
        self.model = model

    def forward(self, img):
        x  = self.model(img,self.cst)
        return x

In [7]:
model.eval()

pmodel = MyModel(model, 'cpu')
pmodel = pmodel
of = 'flat-attention3.onnx'
# input = torch.load("/home/ubuntu/transformer-distillation/tensor_14.pt").unsqueeze(0).cuda()
# input = input.permute(0,2,3,1)
input = torch.rand(1,3,1024,1024).cpu()
torch.onnx.export(pmodel.cpu(), (input.cpu()), of,do_constant_folding=True, export_params=True,input_names = ['input'],output_names = ['output'], verbose=False)

In [22]:
import onnx
from onnx import shape_inference

# Load the ONNX model
model_path = 'channels-last.onnx'
model = onnx.load(model_path)

# Perform shape inference
inferred_model = shape_inference.infer_shapes(model)

# Save the inferred model (optional)
onnx.save(inferred_model, 'channels-last-inferred.onnx')


FileNotFoundError: [Errno 2] No such file or directory: 'channels-last.onnx'

In [ ]:
import onnxruntime as ort
import torch
import numpy as np

In [ ]:
print(sum([p.numel() for p in pmodel.parameters()]) / 1e6)

85.0793


In [ ]:
input = torch.load("/home/ubuntu/transformer-distillation/tensor_14.pt").unsqueeze(0).cuda()
input = input.permute(0,2,3,1)

providers = [
    ('CPUExecutionProvider', {
        'device_id': 0, # The device ID
        # 'trt_max_workspace_size': 4e9, # Maximum workspace size for TensorRT engine (1e9 ≈ 1GB)
        # 'trt_engine_cache_enable': False, # Enable TensorRT engine caching
        # # 'trt_engine_cache_path': str(trt_cache_dir), # Path for TensorRT engine, profile files, and int8 calibration table
        # 'trt_int8_enable': True, # Enable int8 mode in TensorRT
    })
]

# providers = [
#     ('CUDAExecutionProvider')]

sess_options = ort.SessionOptions()
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_DISABLE_ALL
sess_options.enable_mem_pattern = False
sess_options.use_deterministic_compute = True
sess_options.execution_mode = ort.ExecutionMode.ORT_SEQUENTIAL
sess_options.enable_cpu_mem_arena = False



session = ort.InferenceSession("channels-last-inferred.onnx",sess_options,providers=providers)

# Run inference with ONNX Runtime
input_data = input.cpu().numpy().astype(np.float32)
ort_inputs = {session.get_inputs()[0].name: input_data}
# start = torch.cuda.Event(enable_timing=True)
# end = torch.cuda.Event(enable_timing=True)

    # start.record()
onnx_output = session.run(None, ort_inputs)
    # end.record()
    # print(start.elapsed_time(end))
# Compare the outputs

# outim = np.transpose(out[0].data.cpu().numpy(), (1,2,0))
outim = out[0].data.cpu().numpy()
out = Image.fromarray((outim * 255.).astype(np.uint8))
out.save('out_shifted1.png')

/tmp/ipykernel_361958/3462181165.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  input = torch.load("/home/ubuntu/transformer-distillation/tensor_14.pt").unsqueeze(0).cu

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/transformer-distillation/tensor_14.pt'

In [ ]:
import onnx2keras
from onnx2keras import onnx_to_keras
import keras
import onnx


In [ ]:
out = torch.tanh(torch.tensor(onnx_output[0]))
out = out / 2. + .5


In [ ]:
out = Image.fromarray((outim * 255.).astype(np.uint8))
out.save('out_shifted1.png')